This notebook is to further preprocess the property datasets and combine it with external features

In [80]:
import pandas as pd

In [89]:
df_school_prop = pd.read_csv('../data/curated/property_school.csv')
df_cbd_prop = pd.read_csv('../data/curated/property_cbd.csv')
df_sc_prop = pd.read_csv('../data/curated/property_sc.csv')
df_train_prop = pd.read_csv('../data/curated/property_train.csv')


same_attr = ['Unnamed: 0','Unnamed: 0.1','name','cost_text','coordinates','rooms','baths','parking','desc','area']
df = pd.merge(df_school_prop,df_cbd_prop,on=same_attr)
df = pd.merge(df,df_sc_prop,on=same_attr)
df = pd.merge(df,df_train_prop,on=same_attr)

print(df.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15222 entries, 0 to 15221
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               15222 non-null  int64  
 1   Unnamed: 0.1             15222 non-null  object 
 2   name                     15222 non-null  object 
 3   cost_text                15222 non-null  object 
 4   coordinates              15222 non-null  object 
 5   rooms                    15187 non-null  object 
 6   baths                    15183 non-null  object 
 7   parking                  15183 non-null  object 
 8   desc                     15222 non-null  object 
 9   area                     829 non-null    object 
 10  driving_time_to_school   15222 non-null  float64
 11  distance_to_school       15222 non-null  float64
 12  closest_school_name      15222 non-null  object 
 13  closest_school_coor      15222 non-null  object 
 14  driving_time_to_CBD   

In [82]:
# we want to know how many null value each column contains
df.isna().sum()

Unnamed: 0                     0
Unnamed: 0.1                   0
name                           0
cost_text                      0
coordinates                    0
rooms                         35
baths                         39
parking                       39
desc                           0
area                       14393
driving_time_to_school         0
distance_to_school             0
closest_school_name            0
closest_school_coor            0
driving_time_to_CBD            0
distance_to_CBD                0
driving_time_to_sc             0
distance_to_sc                 0
closest_sc_name                0
closest_sc_coor                0
driving_time_to_station        0
distance_to_station            0
closest_station_coor           0
dtype: int64

In [90]:
# remove area and description because there are too many missing values in area, and description does not contain useful determinative information
df = df.drop(['area', 'desc'], axis=1)

# there are few missing values in number of rooms, bathrooms and parking spaces. we tend to remove the ones with no beds and baths provided
# as they are decisive features. However, we fill empty parking with 0

df.dropna(subset = ["rooms","baths"], inplace=True)
df['parking'] = df['parking'].replace(['[]'],"['0 Parking']")
print(df.isna().sum())

Unnamed: 0                 0
Unnamed: 0.1               0
name                       0
cost_text                  0
coordinates                0
rooms                      0
baths                      0
parking                    0
driving_time_to_school     0
distance_to_school         0
closest_school_name        0
closest_school_coor        0
driving_time_to_CBD        0
distance_to_CBD            0
driving_time_to_sc         0
distance_to_sc             0
closest_sc_name            0
closest_sc_coor            0
driving_time_to_station    0
distance_to_station        0
closest_station_coor       0
dtype: int64


In [91]:
# for rooms, baths and parking, we leave only the quantity

rooms = []
baths = []
parking = []
for index, row in df.iterrows():
    rooms.append(row['rooms'][2])
    baths.append(row['baths'][2])
    parking.append(row['parking'][2])
    
df['rooms'] = rooms
df['baths'] = baths
df['parking'] = parking

print(df.head())

   Unnamed: 0                                       Unnamed: 0.1  \
0           0  https://www.domain.com.au/3607-80-a-beckett-st...   
1           1  https://www.domain.com.au/251-350-st-kilda-roa...   
2           2  https://www.domain.com.au/231-299-queen-street...   
3           3  https://www.domain.com.au/5401-464-collins-str...   
4           4  https://www.domain.com.au/4103-80-abeckett-str...   

                                          name       cost_text  \
0  3607/80 A'Beckett Street Melbourne VIC 3000   $3,000 weekly   
1     251/350 St Kilda Road Melbourne VIC 3000  $2800 Per Week   
2      231/299 Queen Street Melbourne VIC 3000       $2,600.00   
3   5401/464 Collins Street Melbourne VIC 3000  $2500 Per Week   
4   4103/80 ABeckett Street Melbourne VIC 3000   Contact Agent   

                  coordinates rooms baths parking  driving_time_to_school  \
0  [-37.8090372, 144.9610905]     2     1       0                   96.04   
1   [-37.831142, 144.9708545]     3     

In [92]:
# format of price is mixed, we tend to change it into price per week. 
# Thinking about the tradition in Australia is usually paying rent by weeks, we also assume that all price listed is rent for a week

import re

price = []
for index, row in df.iterrows():
    
    if re.search(r'\d+', row['cost_text']):
        
        if re.search(r'\d+,*\d+', row['cost_text']) is None:
            price.append('not given')
        
        else:
            price.append(re.findall(r'\$*(\d+,*\d+)\.*\d*',str(df['cost_text'][index]))[0])
    else:
        price.append('not given')

final_price = []
for cost in price:
    if re.search(r',', str(cost)):
        final_price.append(int(re.sub(r',', '', cost)))
    else:
        final_price.append(cost)

df['cost_text'] = final_price

df = df[df['cost_text']!='not given']

print(df.head())


   Unnamed: 0                                       Unnamed: 0.1  \
0           0  https://www.domain.com.au/3607-80-a-beckett-st...   
1           1  https://www.domain.com.au/251-350-st-kilda-roa...   
2           2  https://www.domain.com.au/231-299-queen-street...   
3           3  https://www.domain.com.au/5401-464-collins-str...   
5           5  https://www.domain.com.au/241-299-queen-street...   

                                          name cost_text  \
0  3607/80 A'Beckett Street Melbourne VIC 3000      3000   
1     251/350 St Kilda Road Melbourne VIC 3000      2800   
2      231/299 Queen Street Melbourne VIC 3000      2600   
3   5401/464 Collins Street Melbourne VIC 3000      2500   
5      241/299 Queen Street Melbourne VIC 3000      2400   

                  coordinates rooms baths parking  driving_time_to_school  \
0  [-37.8090372, 144.9610905]     2     1       0                   96.04   
1   [-37.831142, 144.9708545]     3     3       3                  172.97   

In [ ]:
# we now determine which SA2 each property belongs to
import geopandas as gpd
import shapely
import shapely.geometry as geometry

gdf = gpd.read_file('../data/curated/boundary.shp')
gdf = gpd.GeoDataFrame(gdf)

name_list = []
code_list = []

for index, row in df.iterrows():

    lat = float(re.findall(r'\[(-\d+.\d+)',str(row['coordinates']))[0])
    long = float(re.findall(r'(\d+.\d+)\]',str(row['coordinates']))[0])
    property_coor = (long,lat)
    
    flag = 0

    for i in range(len(gdf)):
        if gdf['geometry'][i] is not None:
            if geometry.Point(property_coor).within(geometry.shape(gdf['geometry'][i])):
                flag = 1
                name_list.append(gdf['SA2_NAME21'][i])
                code_list.append(gdf['SA2_CODE21'][i])
                break
        
        if i == len(gdf)-1 and flag == 0:
            code_list.append(0)
            name_list.append('null')


In [ ]:
df['SA2_name'] = name_list
df['SA2_code'] = code_list

In [94]:
# merge the property dataframe with population, income and unemployment rate

df_popu = pd.read_csv('../data/curated/population.csv')

df_popu = df_popu.rename(columns={'SA2 code':'SA2_code'})

year_list = list(range(2001, 2022))
for year in year_list:
    new = str(year) + 'population'
    df_popu = df_popu.rename(columns={str(year):new})

df_popu['SA2_code'] = df_popu['SA2_code'].astype(str)

df_merge = pd.merge(df,df_popu,how='left',on='SA2_code')
df_merge = df_merge.drop(['SA2 name'], axis=1)

print(df_merge.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14767 entries, 0 to 14766
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               14767 non-null  int64  
 1   Unnamed: 0.1             14767 non-null  object 
 2   name                     14767 non-null  object 
 3   cost_text                14767 non-null  object 
 4   coordinates              14767 non-null  object 
 5   rooms                    14767 non-null  object 
 6   baths                    14767 non-null  object 
 7   parking                  14767 non-null  object 
 8   driving_time_to_school   14767 non-null  float64
 9   distance_to_school       14767 non-null  float64
 10  closest_school_name      14767 non-null  object 
 11  closest_school_coor      14767 non-null  object 
 12  driving_time_to_CBD      14767 non-null  float64
 13  distance_to_CBD          14767 non-null  float64
 14  driving_time_to_sc    

In [95]:
df_income = pd.read_csv('../data/curated/income.csv')

df_income = df_income.rename(columns={'SA2':'SA2_code'})
df_income['SA2_code'] = df_income['SA2_code'].astype(str)

df_merge = pd.merge(df_merge,df_income,how='left',on='SA2_code')
df_merge = df_merge.drop(['SA2 NAME'], axis=1)

print(df_merge.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14767 entries, 0 to 14766
Data columns (total 58 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Unnamed: 0                                14767 non-null  int64  
 1   Unnamed: 0.1                              14767 non-null  object 
 2   name                                      14767 non-null  object 
 3   cost_text                                 14767 non-null  object 
 4   coordinates                               14767 non-null  object 
 5   rooms                                     14767 non-null  object 
 6   baths                                     14767 non-null  object 
 7   parking                                   14767 non-null  object 
 8   driving_time_to_school                    14767 non-null  float64
 9   distance_to_school                        14767 non-null  float64
 10  closest_school_name               

In [96]:
df_unemploy = pd.read_csv('../data/curated/unemployment.csv')

df_unemploy = df_unemploy.rename(columns={'SA2 Code (2016 ASGS)':'SA2_code'})
df_unemploy['SA2_code'] = df_unemploy['SA2_code'].astype(str)

df_merge = pd.merge(df_merge,df_unemploy,how='left',on='SA2_code')
df_merge = df_merge.drop(['Statistical Area Level 2 (SA2) (2016 ASGS)', 'Unnamed: 0_y', 'Data Item'], axis=1)

print(df_merge.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14767 entries, 0 to 14766
Data columns (total 69 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Unnamed: 0_x                              14767 non-null  int64  
 1   Unnamed: 0.1                              14767 non-null  object 
 2   name                                      14767 non-null  object 
 3   cost_text                                 14767 non-null  object 
 4   coordinates                               14767 non-null  object 
 5   rooms                                     14767 non-null  object 
 6   baths                                     14767 non-null  object 
 7   parking                                   14767 non-null  object 
 8   driving_time_to_school                    14767 non-null  float64
 9   distance_to_school                        14767 non-null  float64
 10  closest_school_name               

In [97]:
df_merge.to_csv('../data/curated/property_full.csv')